# Chain

链（Chains）通常将大语言模型（LLM）与提示(Prompt)结合在一起，基于此，我们可以对文本或数据进行一系列操作。

链（Chains）可以一次性接受多个输入

例如，我们可以创建一个链，该链接受用户输入，使用提示模板对其进行格式化，然后将格式化的响应传递给LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起来构建更复杂的链。

In [72]:
# set the init envs 
import os, sys
import openai
import langchain
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


openai.api_base = os.environ["OPENAI_API_BASE"] # 换成代理，一定要加v1
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    temperature=0.8,
    model_name="gpt-3.5-turbo",
)
chat = ChatOpenAI(
    temperature=0.0,
    model_name = "gpt-3.5-turbo",
)
# os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# os.environ["WANDB_PROJECT"] = "langchainPlayGround"


def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.2
    )
print(chat)

llm = OpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0
)
print(llm)

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.2 model_kwargs={} openai_api_key='sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5' openai_api_base='https://openkey.cloud/v1' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=None
OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo', 'temperature': 0.0}


d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## 二、大语言模型链

In [6]:
import pandas as pd 

df = pd.read_csv("./data/Data.csv")

df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm = chat, 
                 prompt = prompt,
                 verbose = True
                 )
chain.run(product="Queen Size Sheet Set")



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is the best name to describe     a company that makes Queen Size Sheet Set?

> Finished chain.


'Royal Comfort Linens'

## 三、顺序链
### 3.1 简单顺序链

顺序链（SequentialChains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。

In [15]:
from langchain.chains import SimpleSequentialChain

# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, 
                     prompt=first_prompt,
                     verbose = True,
                     )


# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt,verbose = True,)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [16]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: What is the best name to describe     a company that makes Queen Size Sheet Set?


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.



> Finished chain.
Assistant: A suitable name for a company that makes Queen Size Sheet Sets could be "Royal Rest Linens" or "Regal Dreams Bedding."


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a 20 words description for the following     company:Assistant: A suitable name for a company that makes Queen Size Sheet Sets could be "Royal Rest Linens" or "Regal Dreams Bedding."

> Finished chain.
"Royal Rest Linens: Your go-to destination for luxurious and comfortable Queen Size Sheet Sets fit for royalty."

> Finished chain.


'"Royal Rest Linens: Your go-to destination for luxurious and comfortable Queen Size Sheet Sets fit for royalty."'

### 3.2 顺序链
当只有一个输入和一个输出时，简单顺序链（SimpleSequentialChain）即可实现。当有多个输入或多个输出时，我们则需要使用顺序链（SequentialChain）来实现。

In [18]:
from langchain.chains import SequentialChain


#子链1

# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: 输入：Review 输出： 英文的 Review
chain_one = LLMChain(llm=llm, 
                     prompt=first_prompt, 
                     output_key="English_Review",
                     verbose = True,
                     )


#子链2

# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, 
                     prompt=second_prompt, 
                     output_key="summary",
                     verbose = True,
                     )

#子链3

# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, 
                       prompt=third_prompt, 
                       output_key="language",
                       verbose = True,)


#子链4

# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, 
                      prompt=fourth_prompt, 
                      output_key="followup_message",
                      verbose = True
                      )


#输入：review    
#输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

# # 中文

# #子链1
# # prompt模板 1: 翻译成英语（把下面的review翻译成英语）
# first_prompt = ChatPromptTemplate.from_template(
#     "把下面的评论review翻译成英文:"
#     "\n\n{Review}"
# )
# # chain 1: 输入：Review    输出：英文的 Review
# chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

# #子链2
# # prompt模板 2: 用一句话总结下面的 review
# second_prompt = ChatPromptTemplate.from_template(
#     "请你用一句话来总结下面的评论review:"
#     "\n\n{English_Review}"
# )
# # chain 2: 输入：英文的Review   输出：总结
# chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


# #子链3
# # prompt模板 3: 下面review使用的什么语言
# third_prompt = ChatPromptTemplate.from_template(
#     "下面的评论review使用的什么语言:\n\n{Review}"
# )
# # chain 3: 输入：Review  输出：语言
# chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


# #子链4
# # prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
# fourth_prompt = ChatPromptTemplate.from_template(
#     "使用特定的语言对下面的总结写一个后续回复:"
#     "\n\n总结: {summary}\n\n语言: {language}"
# )
# # chain 4: 输入： 总结, 语言    输出： 后续回复
# chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
#                       output_key="followup_message"
#                      )


# # 对四个子链进行组合
# #输入：review    输出：英文review，总结，后续回复 
# overall_chain = SequentialChain(
#     chains=[chain_one, chain_two, chain_three, chain_four],
#     input_variables=["Review"],
#     output_variables=["English_Review", "summary","followup_message"],
#     verbose=True
# )


In [19]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review to english:

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Can you summarize the following review in 1 sentence:

I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...
Old batch or counterfeit!?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: What language is the following review:

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a follow up response to the following summary in the specified language:

Summary: The reviewer is disappointed with the taste and foam quality of the product, suspecting that it might be an old batch or counterfeit compared to the ones bought in stores.

Language: The language of the review is French.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.



> Finished chain.

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and foam quality of the product, suspecting that it might be an old batch or counterfeit compared to the ones bought in stores.',
 'followup_message': "Cher(e) critique,\n\nNous sommes désolés d'apprendre que vous avez été déçu(e) par le goût et la qualité de la mousse de notre produit. Nous comprenons votre préoccupation quant à la possibilité qu'il s'agisse d'un lot ancien ou contrefait par rapport à ceux achetés en magasin.\n\nNous tenons à vous assurer que nous prenons cette situation très au sérieux. La satisfaction de nos clients est notre priorité absolue et nous nous efforço

## 四、 路由链

到目前为止，我们已经学习了大语言模型链和顺序链。但是，如果我们想做一些更复杂的事情怎么办？

一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

- 路由链（Router Chain）：路由器链本身，负责选择要调用的下一个链
- destination_chains：路由器链可以路由到的链

举一个具体的例子，让我们看一下我们在不同类型的链之间路由的地方，我们在这里有不同的prompt:  

### 4.1 定义提示模板 & 对提示模版进行命名和描述

首先，我们定义提示适用于不同场景下的提示模板。

在定义了这些提示模板后，我们可以为每个模板命名，并给出具体描述。例如，第一个物理学的描述适合回答关于物理学的问题，这些信息将传递给路由链，然后由路由链决定何时使用此子链。

In [144]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    # model_name ="gpt-4",
    temperature=0
    )

#第一个提示适合回答物理问题
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


#第二个提示适合回答数学问题
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


#第三个适合回答历史问题
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""


prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

LLMRouterChain（此链使用 LLM 来确定如何路由事物）

在这里，我们需要一个**多提示链**。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。 但是这只是路由的一种类型，我们也可以在任何类型的链之间进行路由。

这里我们要实现的几个类是大模型路由器链。这个类本身使用语言模型来在不同的子链之间进行路由。这就是上面提供的描述和名称将被使用的地方。

In [145]:
# router chain
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# the default chain for the router
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat, 
                         prompt=default_prompt,
                         verbose=True)

### 4.2 创建默认目标链
除了目标链之外，我们还需要一个默认目标链。这是一个当路由器无法决定使用哪个子链时调用的链。在上面的示例中，当输入问题与物理、数学、历史或计算机科学无关时，可能会调用它。

In [146]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER and Warning: "destination" should be one of the candidate prompt \
names specified below, if the input is not\
well suited for any of the candidate prompts, you should output "default" . 
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>

eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

"""


router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt,
                                       verbose = True)

#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

In [147]:
# # 问题：什么是黑体辐射？
# chain.run("What is black body radiation?")

In [148]:
# # 问题：2+2等于多少？
# chain.run("what is 2 + 2")

In [149]:
# 问题：为什么我们身体里的每个细胞都包含DNA？
# chain.run("Why does every cell in our body contain DNA?")


try:
  # call your Agent as normal
  print(chain.run("Why does every cell in our body contain DNA?"))
  
except Exception as e:
  # any errors will be also logged to Weights & Biases
  print(e)
  pass




> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': 'Why does every cell in our body contain DNA?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Why does every cell in our body contain DNA?

> Finished chain.

> Finished chain.
Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. It serves as a blueprint for the production of specific proteins that determine the characteristics and traits of an organism. Therefore, every cell needs DNA to ensure proper growth, development, and functioning.
